In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline

/Users/dsuniaga/Documents/GitHub/VIXNLP/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
news_df = pd.read_csv('data/finnhub_news.csv')

In [3]:
news_df.head()

,date,symbol,headline,summary,source,url
0,2025-02-20,AAPL,Tracking Ray Dalio's Bridgewater Associates 13...,Bridgewater Associates' Q3 moves: Portfolio va...,SeekingAlpha,https://finnhub.io/api/news?id=f3260216083646e...
1,2025-02-20,AAPL,Madison Sustainable Equity Fund Q4 2024 Invest...,The S&P 500 ended 2024 with a 25% gain for the...,SeekingAlpha,https://finnhub.io/api/news?id=03de51df45863f6...
2,2025-02-20,AAPL,Broadcom Is Threatening AMD's Data Center Oppo...,"Explore Advanced Micro Devices, Inc.'s journey...",SeekingAlpha,https://finnhub.io/api/news?id=a8f77b9fa84d454...
3,2025-02-20,AAPL,Apple's iPhone 16E Strategy,"Nabila Popal, senior research director at the ...",Finnhub,https://finnhub.io/api/news?id=a8bd7172af2bd15...
4,2025-02-20,AAPL,Apple’s iPhone 16e Is a Big Moment for the Com...,Apple’s iPhone 16e Is a Big Moment for the Com...,MarketWatch,https://finnhub.io/api/news?id=1a3f05459e859d2...


In [4]:
pipe = pipeline("text-classification", model="ProsusAI/finbert")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use mps:0


In [11]:
# Convert summaries to list first, then process each with FinBERT
summaries = news_df['summary'].fillna('').astype(str).tolist()

# Create lists to store sentiment scores and labels
sentiment_scores = []
sentiment_labels = []

# Process each summary individually
for summary in summaries:
    result = pipe.predict(summary)[0]
    label = result['label']
    score = result['score']
    
    # Store the label
    sentiment_labels.append(label)
    
    # Calculate the modified score based on label
    if label == 'positive':
        sentiment_scores.append(score)
    elif label == 'negative':
        sentiment_scores.append(score * -1)
    else:  # neutral
        sentiment_scores.append(score / 10)

# Add both the scores and labels to the DataFrame
news_df['summary_sentiment'] = sentiment_scores
news_df['sentiment_label'] = sentiment_labels

In [13]:
# Convert headlines to list first, then process each with FinBERT
headlines = news_df['headline'].fillna('').astype(str).tolist()

# Create lists to store sentiment scores and labels
headline_sentiment_scores = []
headline_sentiment_labels = []

# Process each headline individually
for headline in headlines:
    result = pipe.predict(headline)[0]
    label = result['label']
    score = result['score']
    
    # Store the label
    headline_sentiment_labels.append(label)
    
    # Calculate the modified score based on label
    if label == 'positive':
        headline_sentiment_scores.append(score)
    elif label == 'negative':
        headline_sentiment_scores.append(score * -1)
    else:  # neutral
        headline_sentiment_scores.append(score / 10)

# Add both the scores and labels to the DataFrame
news_df['headline_sentiment'] = headline_sentiment_scores
news_df['headline_sentiment_label'] = headline_sentiment_labels

In [14]:
news_df.head()

,date,symbol,headline,summary,source,url,summary_sentiment,sentiment_label,headline_sentiment,headline_sentiment_label
0,2025-02-20,AAPL,Tracking Ray Dalio's Bridgewater Associates 13...,Bridgewater Associates' Q3 moves: Portfolio va...,SeekingAlpha,https://finnhub.io/api/news?id=f3260216083646e...,-0.627926,negative,0.094340,neutral
1,2025-02-20,AAPL,Madison Sustainable Equity Fund Q4 2024 Invest...,The S&P 500 ended 2024 with a 25% gain for the...,SeekingAlpha,https://finnhub.io/api/news?id=03de51df45863f6...,0.875546,positive,0.093545,neutral
2,2025-02-20,AAPL,Broadcom Is Threatening AMD's Data Center Oppo...,"Explore Advanced Micro Devices, Inc.'s journey...",SeekingAlpha,https://finnhub.io/api/news?id=a8f77b9fa84d454...,0.092798,neutral,-0.906835,negative
3,2025-02-20,AAPL,Apple's iPhone 16E Strategy,"Nabila Popal, senior research director at the ...",Finnhub,https://finnhub.io/api/news?id=a8bd7172af2bd15...,0.087188,neutral,0.088800,neutral
4,2025-02-20,AAPL,Apple’s iPhone 16e Is a Big Moment for the Com...,Apple’s iPhone 16e Is a Big Moment for the Com...,MarketWatch,https://finnhub.io/api/news?id=1a3f05459e859d2...,0.088410,neutral,0.088410,neutral
